# 1. 라이브러리, 데이터 로드

### (1) 라이브러리 임포트

In [39]:
from glob import glob
import pandas as pd
import numpy as np 
from tqdm.auto import tqdm
import cv2
import pickle

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold
import time
import albumentations as albu
from albumentations.pytorch import ToTensorV2

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# IMAGENET_DEFAULT_MEAN = (0.485, 0.456, 0.406)
# IMAGENET_DEFAULT_STD = (0.229, 0.224, 0.225)

### (2) 데이터 로드

In [3]:
all_img_path = np.array(sorted(glob('../data/train/*.png'))) # train data 경로와 파일 이름 저장
test_img_path = np.array(sorted(glob('../data/test/*.png'))) # test data 경로와 파일 이름 저장

In [4]:
train_y = pd.read_csv("../data/train_df.csv") # 파일 이름과 라벨값이 담긴 데이터프레임 로드

train_labels = train_y["label"] # 라벨 값 가져오기

label_unique = sorted(np.unique(train_labels)) # 라벨 값 중 unique한 값 88개 추출
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))} # 88개 값을 숫자로 인코딩 준비

all_label = np.array([label_unique[k] for k in train_labels]) # train data의 label(target) 문자열들을 숫자 형식으로 인코딩

### (3) 파라미터 설정

In [11]:
CFG = {
    'IMG_SIZE':512,
    'EPOCHS':50,
    'LEARNING_RATE':1e-2,
    'BATCH_SIZE':8,
    'SEED':42
}

In [ ]:
rows_train = len(all_img_path) # 주어진 train data의 개수
rows_test = len(test_img_path) # 주어진 test data의 개수
num_trial = 100 # 파라미터 튜닝을 몇 번 진행하는지의 수
splits_hp = 5 # 파라미터 튜닝을 진행할 때의 kfold 수
splits_tr = 5 # 모델 트레이닝을 진행할 때의 kfold 수
basic_seed = 42 # default seed
num_seed_tr = 10 # 트레이닝 seed 개수
sel_seed = 3 # 선택할 seed 개수
num_classes = len(label_unique) # class의 개수

pred_dict = {}
pred_test_dict = {}

# 2. 함수 정의

### Dataset 가져오는 함수

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        # Get image data
        img = cv2.imread(img_path)
        if self.transforms is not None:
            image = self.transforms(image=img)["image"]

        if self.train_mode:
            label = self.label_list[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_path_list)

### Network 정의 함수

In [5]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

### transform 정의 (Data augmentation 시 사용) 

In [6]:
train_transform = albu.Compose([
    albu.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
#     albu.RandomCrop(CFG['IMG_SIZE']-64, CFG['IMG_SIZE']-64),
#     albu.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1,
#                           rotate_limit=30, interpolation=1, border_mode=0,
#                           value=0, p=0.5),
#     albu.HorizontalFlip(p=0.2),
#     albu.VerticalFlip(p=0.2),
#     albu.RandomRotate90(p=0.2),
#     albu.CLAHE(clip_limit=2, p=0.25),
#     albu.Sharpen(p=0.25),
#     albu.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1),
#                                   contrast_limit=(-0.1, 0.1), p=0.25),
#     albu.RandomResizedCrop(height=CFG['IMG_SIZE'], width=CFG['IMG_SIZE'],
#                            scale=(0.5, 1.0), ratio=(0.75, 1.3333333333333333),
#                            interpolation=1, p=1.0),
#     albu.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD, max_pixel_value=512.0, p=1.0),
    ToTensorV2()]
)

valid_transform = albu.Compose([
    albu.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
#     albu.HorizontalFlip(p=0.5),
#     albu.VerticalFlip(p=0.5),
#     albu.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD, max_pixel_value=512.0, p=1.0),
    ToTensorV2()]
)

test_transform = albu.Compose([
    albu.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
#     albu.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD, max_pixel_value=512.0, p=1.0),
    ToTensorV2()]
)

### 모델 학습 함수

In [7]:
def train(model, optimizer, train_loader, vali_loader, scheduler, device):
    model.to(device)

    # Loss Function
    criterion = nn.CrossEntropyLoss().to(device)
    best_score = 0
    
    for epoch in range(1,CFG["EPOCHS"]+1):
        train_pred=[]
        train_y=[]
        model.train()
        train_loss = 0
        for img, label in tqdm(iter(train_loader)):
            
            img, label = img.float().to(device), label.float().to(device)
            
            optimizer.zero_grad()

            # Data -> Model -> Output
            logit = model(img)
            label = label.to(torch.int64)
            loss = criterion(logit, label)

            # backpropagation
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()/len(train_loader)
            train_pred += logit.argmax(1).detach().cpu().numpy().tolist()
            train_y += label.detach().cpu().numpy().tolist()
            
        # Evaluation Validation set
        vali_score = validation(model, vali_loader, device)
        
        # vali_score가 더 이상 커지지 않으면
        if scheduler is not None:
            scheduler.step(vali_score)
        
        print(f'Epoch [{epoch}] Train Score : [{score_function(train_y, train_pred):.5f}] Validation Score : [{vali_score:.5f}]\n')
        
        # Model Saved
        if best_score < vali_score:
            best_score = vali_score
            torch.save(model.state_dict(), '../model/best_model.pth')
            print('Model Saved.')

### 검증 함수

In [8]:
def validation(model, vali_loader, device):
    model.eval() # Evaluation
    logit_list = []
    label_list = []
    with torch.no_grad():
        for img, label in tqdm(iter(vali_loader)):
            img, label = img.float().to(device), label.float().to(device)
            label = label.to(torch.int64)

            logit_list.extend(model(img).argmax(1).detach().cpu().numpy().tolist())
            label_list.extend(label.detach().cpu().numpy().tolist())

    vali_f1_score = score_function(label_list, logit_list)
    return vali_f1_score

### 예측 함수

In [9]:
test_dataset = CustomDataset(test_img_path, None, train_mode=False, transforms=test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def predict(model, test_loader, device):
    model.eval()
    model_pred = []
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            
            img = img.float().to(device)

            pred_logit = model(img).detach().cpu()
            model_pred.extend(pred_logit.tolist())
    return model_pred

def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

# 3. 학습 진행

In [ ]:
kfold = KFold(n_splits=splits_tr, random_state=0, shuffle=True)
cv = np.zeros((rows_train, num_classes))
pred_test = np.zeros((rows_test, num_classes))
for n, (train_idx, val_idx) in enumerate(kfold.split(all_img_path, all_label)):
    
    train_img_path, vali_img_path = all_img_path[train_idx], all_img_path[val_idx]
    train_label, vali_label = all_label[train_idx], all_label[val_idx]
    
    # Get Dataloader
    train_dataset = CustomDataset(train_img_path.tolist(), train_label.tolist(), train_mode=True, transforms=train_transform)
    train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

    vali_dataset = CustomDataset(vali_img_path.tolist(), vali_label.tolist(), train_mode=True, transforms=valid_transform)
    vali_loader = DataLoader(vali_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    
    model = Network().to(device)
    optimizer = torch.optim.SGD(params = model.parameters(), lr = CFG["LEARNING_RATE"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max', patience=3, factor=0.2)
    
    train(model, optimizer, train_loader, vali_loader, scheduler, device)
    
    checkpoint = torch.load('../model/best_model.pth')
    model = Network().to(device)
    model.load_state_dict(checkpoint)
    
    vali_dataset = CustomDataset(vali_img_path.tolist(), vali_label.tolist(), train_mode=False, transforms=test_transform)
    vali_loader = DataLoader(vali_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    
    cv[val_idx, :] = predict(model, vali_loader, device)
    pred_test += np.array(predict(model, test_loader, device)) / splits_tr
    print(f"Fold {n+1} Score: {score_function(vali_label, cv[val_idx].argmax(axis=1))}")
    
pred_dict['eff_b0'+str(seed)] = cv
pred_test_dict['eff_b0'+str(seed)] = pred_test

# 4. 학습 결과 저장

In [36]:
def sort_dict(model, pred_dict, pred_test_dict):
    pred_dict_local = {}
    for key, value in pred_dict.items():
        if model in key:
            pred_dict_local[key]=value

    pred_test_dict_local = {}
    for key, value in pred_test_dict.items():
        if model in key:
            pred_test_dict_local[key]=value

    pred_dict_new_local = dict(sorted(
        pred_dict_local.items(), 
        key=lambda x:score_function((all_label), np.argmax(list(x[1]), axis=1)), reverse=False)[:5])
    pred_test_dict_new_local = {}
    for key, value in pred_dict_new_local.items():
        pred_test_dict_new_local[key]=pred_test_dict_local[key]
        
    return pred_dict_new_local, pred_test_dict_new_local

In [37]:
def save_dict(model, pred_dict, pred_test_dict):
    with open('../pickle/pred_dict_'+model+'.pickle', 'wb') as fw:
        pickle.dump(pred_dict, fw)
    with open('../pickle/pred_test_dict_'+model+'.pickle', 'wb') as fw:
        pickle.dump(pred_test_dict, fw)

In [43]:
pred_dict_efficientb0, pred_test_dict_efficientb0 = sort_dict('efficientb0', pred_dict, pred_test_dict)
save_dict('efficientb0', pred_dict_efficientb0, pred_test_dict_efficientb0)

# 5. 제출

In [32]:
label_decoder = {val:key for key, val in label_unique.items()}
pred_final = [label_decoder[result] for result in pred_test.argmax(1)]

In [33]:
from datetime import datetime
submission = pd.read_csv('../data/sample_submission.csv')
submission["label"] = pred_final
submission_time = datetime.today().strftime('%Y-%m-%d-%M-%S')
submission.to_csv(f'../submission/{submission_time}.csv', index = False)

In [34]:
submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-misplaced
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good
